<a href="https://colab.research.google.com/github/BhaskarFysik/AlgoShared/blob/main/bh_emacross_backtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.4 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [ ]:
pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install pandas_ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 115 kB 11.3 MB/s 
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218923 sha256=7fd9c08ab601a8911842189fef9ece2eac2953d5f01dec97ff4bfd0d68c269e7
  Stored in directory: /root/.cache/pip/wheels/0b/81/f0/cca85757840e4616a2c6b9fe12569d97d324c27cac60724c58
Successfully built pandas-ta


In [ ]:
import yfinance as yf
import pandas_ta as ta
import pandas as pd
import copy
import numpy as np
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)


def symbols_backtesting(symbol_list):
    all_trades = []
    for symbol in symbol_list:
        df = yf.Ticker(f"{symbol}.NS").history(period="5y", interval="1d")
        df["MA_20"] = ta.ema(close=df["Close"], length=20)
        df["MA_50"] = ta.ema(close=df["Close"], length=50)
        df["RSI_14"] = ta.rsi(close=df["Close"], length=14)
        # df["ATR_14"] = ta.ATR(df["High"], df["Low"], df["Close"], timeperiod=14)
        # df["Upper_Band"], df["Middle_Band"], df["Lower_Band"] = talib.BBANDS(df["Close"], timeperiod=20,
        #                                                                      nbdevup=2,nbdevdn=2)

        trade = {"Symbol": None, "Buy/Sell": None, "Entry": None, "Entry Date": None, "Exit": None, "Exit Date": None}
        # print(df)
        position = None
        for i in df.index[49:]:
            if df["MA_20"][i] > df["MA_50"][i] and df["RSI_14"][i] > 70 and position != "Buy":
                if trade["Symbol"] is not None:
                    trade["Exit"] = df["Close"][i]
                    trade["Exit Date"] = i
                    all_trades.append(copy.deepcopy(trade))
                if position is not None:
                    trade["Symbol"] = symbol
                    trade["Buy/Sell"] = "Buy"
                    trade["Entry"] = df["Close"][i]
                    trade["Entry Date"] = i
                position = "Buy"
            if df["MA_20"][i] < df["MA_50"][i] and df["RSI_14"][i] < 30 and position != "Sell":
                if trade["Symbol"] is not None:
                    trade["Exit"] = df["Close"][i]
                    trade["Exit Date"] = i
                    all_trades.append(copy.deepcopy(trade))
                if position is not None:
                    trade["Symbol"] = symbol
                    trade["Buy/Sell"] = "Sell"
                    trade["Entry"] = df["Close"][i]
                    trade["Entry Date"] = i
                position = "Sell"
    return all_trades


symbol_list = ['ACC', 'ADANIGREEN', 'ATGL', 'ADANITRANS', 'AMBUJACEM', 'DMART', 'BAJAJHLDNG', 'BANDHANBNK', 'BANKBARODA', 'BERGEPAINT', 'BEL', 'BIOCON', 'BOSCHLTD', 'CHOLAFIN', 'COLPAL', 'DLF', 'DABUR', 'NYKAA', 'GAIL', 'GLAND', 'GODREJCP', 'HDFCAMC', 'HAVELLS', 'HAL', 'ICICIGI', 'ICICIPRULI', 'IOC', 'IRCTC', 'INDUSTOWER', 'NAUKRI', 'INDIGO', 'LTI', 'LICI', 'MARICO', 'MPHASIS', 'MUTHOOTFIN', 'PAYTM', 'PIIND', 'PIDILITIND', 'PGHH', 'SBICARD', 'SRF', 'MOTHERSON', 'SHREECEM', 'SIEMENS', 'TATAPOWER', 'TORNTPHARM', 'MCDOWELL-N', 'VEDL', 'ZOMATO']

data = symbols_backtesting(symbol_list)
if data:
    risk_percent = 20/100
    df = pd.DataFrame(data)
    df["P/L"] = np.where(df["Buy/Sell"] == "Buy", (100*(df["Exit"] - df["Entry"])/df["Entry"])*risk_percent,
                         (100*(df["Entry"] - df["Exit"])/df["Entry"])*risk_percent)
    df = df[df["Buy/Sell"] == "Buy"].reset_index(drop=True)

    #df["Probability"] = 100*(np.where(df["P/L"] > 0, 1, 0).cumsum())/(np.where(df["P/L"] != np.NaN, 1, 0).cumsum())
    df["Return"] = df["P/L"].cumsum()
    df["Drawdown"] = df["Return"] - (df["Return"].cummax().apply(lambda x: x if x > 0 else 0))

    print(df)
    df.to_csv("backtest_macross_rsi_1D.csv")
else:
    print("No Trades")

         Symbol Buy/Sell         Entry                Entry Date  \
0           ACC      Buy   1416.175049 2018-07-27 00:00:00+05:30   
1           ACC      Buy   1397.589355 2020-07-30 00:00:00+05:30   
2           ACC      Buy   2232.899902 2022-04-11 00:00:00+05:30   
3    ADANIGREEN      Buy     44.500000 2019-04-16 00:00:00+05:30   
4    ADANIGREEN      Buy    193.850006 2020-04-16 00:00:00+05:30   
5          ATGL      Buy    160.825089 2019-11-29 00:00:00+05:30   
6          ATGL      Buy    135.911484 2020-06-03 00:00:00+05:30   
7    ADANITRANS      Buy    203.850006 2018-08-27 00:00:00+05:30   
8    ADANITRANS      Buy    209.300003 2018-11-15 00:00:00+05:30   
9    ADANITRANS      Buy    242.550003 2020-06-23 00:00:00+05:30   
10    AMBUJACEM      Buy    208.260162 2018-08-01 00:00:00+05:30   
11    AMBUJACEM      Buy    201.990829 2020-07-30 00:00:00+05:30   
12        DMART      Buy   1559.750000 2018-12-12 00:00:00+05:30   
13        DMART      Buy   1446.599976 2019-07-0

# New section